In [1]:
# --- 匯入套件 ---
import os
import glob
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
# --- Step 1: 讀取大量 CSV ---
csv_folder = "./dance_csv/"  # 你的 CSV 資料夾
csv_files = glob.glob(os.path.join(csv_folder, "*.csv"))

all_poses = []

def parse_point(s):
    s = s.strip("()")
    parts = [float(p.strip(" '")) for p in s.split(",")]
    return parts

for csv_path in csv_files:
    df = pd.read_csv(csv_path)
    pose_cols = [c for c in df.columns if c != "frame"]
    for i, row in df.iterrows():
        pose = []
        for c in pose_cols:
            pose += parse_point(row[c])
        all_poses.append(pose)

all_poses = np.array(all_poses)  # shape: (total_frames, 99)
print("骨架資料 shape:", all_poses.shape)

骨架資料 shape: (271194, 99)


In [3]:
# --- Step 2: 標準化 ---
scaler = StandardScaler()
all_poses = scaler.fit_transform(all_poses)

In [4]:
# --- Step 3: 建立 Dataset / DataLoader ---
class PoseDataset(Dataset):
    def __init__(self, data, seq_len=10):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return len(self.data) - self.seq_len
    def __getitem__(self, idx):
        seq = self.data[idx:idx+self.seq_len]
        return seq, seq  # autoencoder

seq_len = 10
dataset = PoseDataset(all_poses, seq_len)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [5]:
# --- Step 4: 定義 LSTM Autoencoder ---
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim=99, hidden_dim=128, latent_dim=32, num_layers=1):
        super().__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc_enc = nn.Linear(hidden_dim, latent_dim)
        self.fc_dec = nn.Linear(latent_dim, hidden_dim)
        self.decoder = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        enc_out, (h_n, c_n) = self.encoder(x)
        # 取最後 timestep 的 hidden state
        latent = self.fc_enc(enc_out[:, -1, :])
        # decoder
        dec_input = self.fc_dec(latent).unsqueeze(1).repeat(1, x.size(1), 1)
        dec_out, _ = self.decoder(dec_input)
        return dec_out, latent

device = "cuda" if torch.cuda.is_available() else "cpu"
model = LSTMAutoencoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [6]:
# --- Step 5: 訓練 Autoencoder ---
epochs = 30
for epoch in range(epochs):
    total_loss = 0
    for batch, _ in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        recon, latent = model(batch)
        loss = loss_fn(recon, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/30, Loss: 0.3953
Epoch 2/30, Loss: 0.3629
Epoch 3/30, Loss: 0.3560
Epoch 4/30, Loss: 0.3524
Epoch 5/30, Loss: 0.3498
Epoch 6/30, Loss: 0.3479
Epoch 7/30, Loss: 0.3465
Epoch 8/30, Loss: 0.3454
Epoch 9/30, Loss: 0.3445
Epoch 10/30, Loss: 0.3437
Epoch 11/30, Loss: 0.3429
Epoch 12/30, Loss: 0.3423
Epoch 13/30, Loss: 0.3416
Epoch 14/30, Loss: 0.3411
Epoch 15/30, Loss: 0.3408
Epoch 16/30, Loss: 0.3402
Epoch 17/30, Loss: 0.3399
Epoch 18/30, Loss: 0.3395
Epoch 19/30, Loss: 0.3392
Epoch 20/30, Loss: 0.3389
Epoch 21/30, Loss: 0.3386
Epoch 22/30, Loss: 0.3383
Epoch 23/30, Loss: 0.3380
Epoch 24/30, Loss: 0.3377
Epoch 25/30, Loss: 0.3376
Epoch 26/30, Loss: 0.3374
Epoch 27/30, Loss: 0.3371
Epoch 28/30, Loss: 0.3370
Epoch 29/30, Loss: 0.3367
Epoch 30/30, Loss: 0.3365


In [7]:
# --- Step 6: 生成 latent 向量 ---
model.eval()
all_latents = []
with torch.no_grad():
    for batch, _ in dataloader:
        batch = batch.to(device)
        _, latent = model(batch)
        all_latents.append(latent.cpu().numpy())
all_latents = np.concatenate(all_latents, axis=0)
print("Latent shape:", all_latents.shape)

Latent shape: (271184, 32)


In [8]:
# --- Step 7: KMeans 聚類成符號 ---
num_symbols = 26  # 對應 A-Z
kmeans = KMeans(n_clusters=num_symbols, random_state=42, n_init=10)
labels = kmeans.fit_predict(all_latents)
symbols = [chr(65 + l) for l in labels]  # A-Z
sequence = "".join(symbols)
print("符號序列預覽：", sequence[:200])

符號序列預覽： EDOQPVZTMSUYCCMWCSRPBNPTYIBCBBYYDYHHZPGDSAWKHOCWOYVCSLKBGAROCCSFGBZPHBVQCIHDRBVHPYYFVPZJFGGHBTAQNDCNNPCBYYBCYBRLVVADQCEDYKGNRYBQCLBMSSHQRIFSPZAAPVTPHWRFZWARABBRENLAWNLRKGDLZGSCHAZFVHRVMTFDZCTRNXYFDBIS


In [9]:
# --- Step 8: 儲存為 JSON ---
output_path = "symbol_sequences07.json"
with open(output_path, "w") as f:
    json.dump({"symbols": sequence}, f)
print("✅ 已儲存符號序列：", output_path)

✅ 已儲存符號序列： symbol_sequences.json
